In [1]:
import pandas as pd

import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
import joblib
import json

from ezkl import export

## Read the Data

In [2]:
data = pd.read_csv('clasificationTrainData-without-Yield.csv.gz', compression='gzip')
print(data.shape)
data.head(2)

(12696, 71)


,target,apy-Var1Day,tvlUsd-Var1Day,apy-chain-mean-Var1Day,tvlUsd-chain-mean-Var1Day,apy-protocol-mean-Var1Day,tvlUsd-protocol-mean-Var1Day,apy-token-mean-Var1Day,tvlUsd-token-mean-Var1Day,apy-chain-max-Var1Day,...,apy-protocol-mean-Var30Day,tvlUsd-protocol-mean-Var30Day,apy-token-mean-Var30Day,tvlUsd-token-mean-Var30Day,apy-chain-max-Var30Day,apy-protocol-max-Var30Day,apy-token-max-Var30Day,tvlUsd-chain-sum-Var30Day,tvlUsd-protocol-sum-Var30Day,tvlUsd-token-sum-Var30Day
0,0,0.007886,-0.002881,0.384474,0.012158,0.008700,0.002458,0.074924,0.012354,0.619913,...,0.528504,0.212078,1.080723,0.087964,2.729201,0.353595,1.641604,0.040766,0.212078,0.087964
1,0,0.000000,-0.002264,0.202158,0.034465,-0.057477,0.015966,0.084087,0.015578,0.021917,...,0.511465,0.018395,4.537764,-0.006988,0.321799,0.791096,11.122229,0.097451,0.018395,-0.006988


## Split, scale and create Tensors

In [3]:
X = data.iloc[:, 1:11]
y = data.iloc[:, 0]

scaler = MinMaxScaler()
X = scaler.fit_transform(X.values)
joblib.dump(scaler, 'X_scaler.joblib')

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

X_train_tensor = torch.tensor(X_train, dtype=torch.float)
y_train_tensor = torch.tensor(y_train.values, dtype=torch.float)
X_test_tensor = torch.tensor(X_test, dtype=torch.float)
y_test_tensor = torch.tensor(y_test.values, dtype=torch.float)

## Define Model

In [10]:
class BinaryClassifier(nn.Module):
    def __init__(self, hidden_size, input_size):
        super().__init__()
        self.fc1 = nn.Linear(input_size, hidden_size)
        self.relu1 = nn.ReLU()
        self.fc2 = nn.Linear(hidden_size, hidden_size)
        self.relu2 = nn.ReLU()
        self.fc3 = nn.Linear(hidden_size, hidden_size)
        self.relu3 = nn.ReLU()
        self.fc4 = nn.Linear(hidden_size, hidden_size)
        self.relu4 = nn.ReLU()
        self.fc5 = nn.Linear(hidden_size, hidden_size)
        self.relu5 = nn.ReLU()
        self.fc6 = nn.Linear(hidden_size, hidden_size)
        self.relu6 = nn.ReLU()
        self.fc7 = nn.Linear(hidden_size, hidden_size)
        self.relu7 = nn.ReLU()
        self.fc8 = nn.Linear(hidden_size, hidden_size)
        self.relu8 = nn.ReLU()
        self.fc9 = nn.Linear(hidden_size, hidden_size)
        self.relu9 = nn.ReLU()
        self.fc10 = nn.Linear(hidden_size, hidden_size)
        self.relu10 = nn.ReLU()
        self.fc11 = nn.Linear(hidden_size, hidden_size)
        self.relu11 = nn.ReLU()
        self.output = nn.Linear(hidden_size, 1)
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        x = self.fc1(x)
        x = self.relu1(x)
        x = self.fc2(x)
        x = self.relu2(x)
        x = self.fc3(x)
        x = self.relu3(x)
        x = self.fc4(x)
        x = self.relu4(x)
        x = self.fc5(x)
        x = self.relu5(x)
        x = self.fc6(x)
        x = self.relu6(x)
        x = self.fc7(x)
        x = self.relu7(x)
        x = self.fc8(x)
        x = self.relu8(x)
        x = self.fc9(x)
        x = self.relu9(x)
        x = self.fc10(x)
        x = self.relu10(x)
        x = self.fc11(x)
        x = self.relu11(x)
        x = self.output(x)
        x = self.sigmoid(x)
        return x

## Train the same model in 1000 epochs to update weights

In [11]:
epochs = 1000
hidden_size = 10
learning_rate = 0.001
input_size = X_train_tensor.shape[1]

# Initialize the model
model = BinaryClassifier(hidden_size, input_size)

# Define the loss function and the optimizer
criterion = nn.BCELoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate, weight_decay=1e-5)

for epoch in range(epochs):
    model.train()  # Set the model to training mode

    # Forward pass
    outputs = model(X_train_tensor).squeeze()
    loss = criterion(outputs, y_train_tensor)

    # Backward and optimize
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

## Export the trained model

In [12]:
export(model, input_shape=[input_size])

============== Diagnostic Run torch.onnx.export version 2.0.1+cpu ==============
verbose: False, log level: Level.ERROR
======================= 0 NONE 0 NOTE 0 WARNING 0 ERROR ========================



### Save the model to use it in the other script

In [13]:
model_scripted = torch.jit.script(model) # Export to TorchScript
model_scripted.save('linear-pytorch.pt') # Save

____________